### CNN dimensions stuff

In [4]:
data=[]
for x in range(5):
    data.append(x)
n = len(data)
for z in range(10):
    data.append(z)
m = len(data[n:])

print(n,m, len(data))

5 10 15


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
import pandas as pd
import numpy as np
import sys,os,re,csv,pathlib
#Allows relative imports
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
#import tensorflow as tf
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score

import warnings
warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2


ModuleNotFoundError: No module named 'tensorflow'

In [5]:
from src.models import *
deepcat_cnn(12)

deepcat_cnn(
  (conv1): Conv2d(1, 8, kernel_size=(15, 2), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=(1, 2), stride=(1, 1), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 16, kernel_size=(1, 2), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=(1, 2), stride=(1, 1), padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=128, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=2, bias=True)
  (dropout): Dropout(p=0.4, inplace=False)
)

# c1 = nn.Conv2d(1, 8, kernel_size=(15,2))
m1 = nn.MaxPool2d(kernel_size = (1,2),stride = (1,1))
c2 = nn.Conv2d(8, 16, kernel_size = (1,2))
m2 = nn.MaxPool2d(kernel_size = (1,2),stride = (1,1))

cnn = nn.Sequential(c1,m1,c2,m2)

In [7]:
for z in range(12,13):
    print("####\n\nUSING LENGTH = ",z)
    x = torch.rand((1000,1,15,z),dtype=torch.float32)
    print("Input: ",x.shape)
    conv1=c1(x)
    print("After Conv1:", conv1.shape)
    pool1=m1(conv1)
    print("After Pool1:", pool1.shape)
    conv2=c2(pool1)
    print("After Conv2:", conv2.shape)
    pool2=m2(conv2)
    print("After Pool2:", pool2.shape)
    xx= (z-4)*16
    #if z<15:
    #    xx = 16*2
    #elif z>=15:
    #    xx = 16*3
    lin = nn.Linear(xx,10)

    dense1= lin(pool2.view(-1,xx))
    print("Dense",dense1.shape)
    lin2= nn.Linear(10,2)
    dense2= lin2(dense1)
    print("Output",dense2.shape)
    #print("soft",dense1.softmax(1))
    #print("arg",dense1.argmax(1))
    #print("####\n\n")


####

USING LENGTH =  12
Input:  torch.Size([1000, 1, 15, 12])
After Conv1: torch.Size([1000, 8, 1, 11])
After Pool1: torch.Size([1000, 8, 1, 10])
After Conv2: torch.Size([1000, 16, 1, 9])
After Pool2: torch.Size([1000, 16, 1, 8])
Dense torch.Size([1000, 10])
Output torch.Size([1000, 2])


In [8]:
from src.models import deepcat_cnn
for l in range(12,17):
    print("###################\n For L = {}".format(l))
    model = deepcat_cnn(l)
    feat = torch.empty((100,1,15,l))
    out = model(feat)
    print("###################\n")

###################
 For L = 12
input torch.Size([100, 1, 15, 12])
after conv1 torch.Size([100, 8, 1, 11])
after pool1 torch.Size([100, 8, 1, 10])
After conv2 torch.Size([100, 16, 1, 9])
After pool2 torch.Size([100, 16, 1, 8])
Before reshape torch.Size([100, 16, 1, 8])
reshaped torch.Size([100, 128])
###################

###################
 For L = 13
input torch.Size([100, 1, 15, 13])
after conv1 torch.Size([100, 8, 1, 12])
after pool1 torch.Size([100, 8, 1, 11])
After conv2 torch.Size([100, 16, 1, 10])
After pool2 torch.Size([100, 16, 1, 9])
Before reshape torch.Size([100, 16, 1, 9])
reshaped torch.Size([100, 144])
###################

###################
 For L = 14
input torch.Size([100, 1, 15, 14])
after conv1 torch.Size([100, 8, 1, 13])
after pool1 torch.Size([100, 8, 1, 12])
After conv2 torch.Size([100, 16, 1, 11])
After pool2 torch.Size([100, 16, 1, 10])
Before reshape torch.Size([100, 16, 1, 10])
reshaped torch.Size([100, 160])
###################

###################
 For L 

### Parsing/AA index stuff/encoding

In [49]:
AAs=np.array(list('WFGAVILMPYSTNQCKRHDE'))
curPath=os.getcwd()
##AAidx_file='AAindexNormalized.txt' ## AA index reached AUC about 61% for L=14. Worse than AdaBoost
##AAidx_file='AtchleyFactors.txt'  ## Atchley factors work worse than using 544 AA index
AAidx_file='AAidx_PCA.txt' ## works like a charm!!!
gg=open(AAidx_file)
AAidx_Names=gg.readline().strip().split('\t') # Get PC1,... PC15
AAidx_Dict={} # Gets the values for each of the 15 features
i = 0
for ll in gg.readlines():
    if i ==4:
        print(ll)
    
    ll=ll.strip().split('\t')
    AA=ll[0]
    tag=0
    vv=[]
    for xx in ll[1:]:
        vv.append(float(xx))
    if tag==1:
        i+=1
        continue
    i+=1
    AAidx_Dict[AA]=vv
    
Nf=len(AAidx_Dict['C']) #could've just written 15...

pat=re.compile('[\\*_XB]')  ## non-productive CDR3 patterns


C	-8.36918251369427	8.3031934498954	-6.6196694564762	13.8734139891373	8.59531323791867	9.79914818190207	1.26911186662177	-4.63752901352054	-0.983921565990354	4.3634243590938	2.34365256090458	0.343445840449375	1.18290876288827	0.0132558803925318	-0.696621502763574



In [5]:
import pickle
#Save the output of the mapping AAidx_dict in the beginning of DeepCAT.py
with open('AAidx_dict.pkl', 'wb') as f: 
    pickle.dump(AAidx_Dict, f)

In [50]:
np.set_printoptions(precision=3)
AAs=np.array(list('WFGAVILMPYSTNQCKRHDE'))

def OneHotEncoding(Seq):
    Seq_aa = list(Seq) 
    Ns=len(Seq_aa)
    OHE=np.zeros([20,Ns])
    for ii in range(Ns):
        aa=Seq_aa[ii]
        vv=np.where(AAs==aa)
        OHE[vv,ii]=1
    OHE=OHE.astype(np.float32)
    return OHE, vv 
#So in X (row/index) we have the amino acid 
#And in Y (columns) we have the position in the sequence
seq = 'CASSYSTRGGSPLHF'
temp, vv = OneHotEncoding(seq)
print(temp.dtype,"\n", temp.shape,"\n",temp,"\n",type(temp))

float32 
 (20, 15) 
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]] 
 <class 'numpy.nd

In [58]:
print(oneHot_encoding('CASSYSGGSPLHF'))

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        

In [ ]:
#Nf = number features = 15
Nf=15
def AAindexEncoding(Seq):
    Ns=len(Seq)
    AAE=np.zeros([Ns, Nf])
    for kk in range(Ns):
        ss=Seq[kk]
        #print(ss, kk)
        AAE[kk,]=AAidx_Dict[ss]
    AAE=np.transpose(AAE.astype(np.float32))
    return AAE
seq = 'CASSYSGGSPLHF'
temp = AAindexEncoding(seq)
print(temp,temp.shape)
#print(temp.dtype,"\n", temp.shape,"\n",temp,"\n",type(temp))

In [61]:
llt = torch.tensor([12, 13, 14, 12, 12, 15, 15, 15, 12])

ll = 12
ll1 = 15
x= torch.tensor([1]*len(torch.where(llt==ll)[0])+[0]*len(torch.where(llt==ll1)[0]))
x


tensor([1, 1, 1, 1, 0, 0, 0])

In [71]:
x.size()

torch.Size([7])

#### draft for generate_features_labels

In [32]:
# get current directory 
path = os.getcwd() 
print("Current Directory", path) 
  
# prints parent directory 
par = os.path.abspath(os.path.join(path, os.pardir))


Current Directory C:\Users\richi\Documents\EPFL\Master\PDM\code\DeepTCR_PyTorch\notebooks


['NormalCDR3.txt',
 'NormalCDR3_test.txt',
 'readme.md',
 'TumorCDR3.txt',
 'TumorCDR3_test.txt']

In [ ]:
import pickle
with open('../AAidx_dict.pkl', 'rb') as f: 
    AAidx_Dict = pickle.load(f) 
def aaindex_encoding(seq):
	"""Encodes the AA indices to a given sequence"""
	n_aa = len(seq)
	temp = np.zeros([n_aa, 15], dtype=np.float32)
	for idx in range(n_aa):
		aa = seq[idx]
		temp[idx] = AAidx_Dict[aa]
	temp = np.transpose(temp)
	aa_encoding = torch.from_numpy(temp)
	return aa_encoding

In [11]:
def read_sequences(filename):
	"""read sequences from a txt, and """
	if '.txt' not in filename:
		print("Non .txt file given, exiting")
		return
		
	data = []
	with open(filename, 'r') as f:
		for line in f.readlines():
			seq = line.strip()
			if not seq.startswith('C') or not seq.endswith('F'):continue
			data.append(seq)
	return np.array(data)


In [12]:
# get current directory 
path = os.getcwd() 
print("Current Directory", path) 
  
# prints parent directory 
par = os.path.abspath(os.path.join(path, os.pardir))

for filename in os.listdir(par+'/TrainingData/'):
    if '.txt' in filename:
        test = read_sequences(par+'/TrainingData/'+filename)
#DRAFT FOR GENERATE DATA        
tumor_sequences = np.array(test[0:1000])
normal_sequences = np.array(test[1000:5000])
seqlens_tumor = np.array([len(seqs) for seqs in tumor_sequences]) 
seqlens_normal = np.array([len(seqs) for seqs in normal_sequences])
length = 12
mask_tumor = np.where(seqlens_tumor==length)[0]
mask_normal = np.where(seqlens_normal==length)[0]
#Reusing the code from DeepCAT for Labels
labels = torch.tensor(([1]*len(mask_tumor)+[0]*len(mask_normal)), dtype=torch.int64)
len(labels)
data = []
PAT = re.compile('[\\*_XB]')  ## non-productive CDR3 patterns
for seqs in tumor_sequences[mask_tumor]:
    if len(PAT.findall(seqs))>0:continue #Skipping a sequence if it matches an unwanted CDR3 pattern 
    data.append(aaindex_encoding(seqs))

for seqs in normal_sequences[mask_normal]:
    if len(PAT.findall(seqs))>0:continue #Skipping a sequence if it matches an unwanted CDR3 pattern 
    data.append(aaindex_encoding(seqs))
data = torch.stack(data)

feature_dict, label_dict = {}, {}
features = {'x':data, 'length':length}
feature_dict[length] = features
label_dict[length] = labels 

Current Directory C:\Users\richi\Documents\EPFL\Master\PDM\code\DeepTCR_PyTorch\notebooks


NameError: name 'aaindex_encoding' is not defined

### TESTING UNSQUEEZE DIM ISSUES


In [40]:
# get current directory 
import pickle
path = os.getcwd() 
print("Current Directory", path) 
#Loading the pre-set Dictionary with values from PCA1-15 AA indices
with open('../AAidx_dict.pkl', 'rb') as f: 
    AAidx_Dict = pickle.load(f) 
  
# prints parent directory 
par = os.path.abspath(os.path.join(path, os.pardir))

AAs = np.array(list('WFGAVILMPYSTNQCKRHDE')) #Constant list of amino acids
PAT = re.compile('[\\*_XB]')  ## non-productive CDR3 patterns

def aaindex_encoding(seq, device):
    """Encodes the AA indices to a given sequence"""
    n_aa = len(seq)
    temp = np.zeros([n_aa, 15], dtype=np.float32)
    for idx in range(n_aa):
        aa = seq[idx]
        temp[idx] = AAidx_Dict[aa]
    temp = np.transpose(temp)
    aa_encoding = torch.from_numpy(temp)
    aa_encoding = aa_encoding.unsqueeze(0)
    if device == torch.device('cuda'):
        aa_encoding = aa_encoding.to(device)
    return aa_encoding

def generate_features_labels(tumor_sequences, normal_sequences, device):
    """For each CDR3 dataset (tumor and normal) sequences, get the feature vectors and labels"""
    
    #Normally, sequences are extracted as lists, but maybe I can modify something in read_sequences to return array instead of list
    if type(tumor_sequences)!=np.ndarray : tumor_sequences = np.array(tumor_sequences)
    if type(tumor_sequences)!=np.ndarray : normal_sequences = np.array(normal_sequences)

    #length of each datapoint (sequence)
    seqlens_tumor = np.array([len(seqs) for seqs in tumor_sequences]) 
    seqlens_normal = np.array([len(seqs) for seqs in normal_sequences])

    feature_dict, label_dict = {}, {}
    #Only keep sequences with length 12 to 16
    for length in range(12,17):
        #Using numpy to create mask for fancy indexing, converting to tensors later
        mask_tumor = np.where(seqlens_tumor==length)[0]
        mask_normal = np.where(seqlens_normal==length)[0]
        #Reusing the code from DeepCAT for Labels
        labels = torch.tensor(([1]*len(mask_tumor)+[0]*len(mask_normal)), dtype=torch.int64)
        data = []

        for seqs in tumor_sequences[mask_tumor]:
            if len(PAT.findall(seqs))>0:continue #Skipping a sequence if it matches an unwanted CDR3 pattern 
            data.append(aaindex_encoding(seqs, device))

        for seqs in normal_sequences[mask_normal]:
            if len(PAT.findall(seqs))>0:continue #Skipping a sequence if it matches an unwanted CDR3 pattern 
            data.append(aaindex_encoding(seqs, device))

        data = torch.stack(data) #Stack a list of tensors into a single tensor
        if device == torch.device('cuda'):
            data = data.to(device)
        features = {'x':data, 'length':length}
        feature_dict[length] = features
        label_dict[length] = labels 

    return feature_dict, label_dict

for filename in os.listdir(par+'/TrainingData/'):
    if '.txt' in filename:
        test = read_sequences(par+'/TrainingData/'+filename)
feats, lab = generate_features_labels(tumor_sequences, normal_sequences, 'cuda')
feats[12]['x'].shape

Current Directory C:\Users\richi\Documents\EPFL\Master\PDM\code\DeepTCR_PyTorch\notebooks


torch.Size([487, 1, 15, 12])